In [4]:
from xml.dom import minidom
import glob, os
#import nltk
from collections import Counter
import re

import liwc

In [5]:
def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

parse, category_names = liwc.load_token_parser('data/liwc.dic')

In [40]:
# buscando categorias do liwc
with open('data/liwc.dic') as f:
    linhas = f.readlines()
    
liwcCats = []
for cont, linha in enumerate(linhas):
    linha = linha.strip("\n")
    if linha == "%":
        if cont > 2:
            break
        else:
            continue
    it = linha.split("\t")[1]
    #print(it)
    liwcCats.append(it)
liwcCats.sort()
print(liwcCats)

['achieve', 'adverb', 'affect', 'anger', 'anx', 'article', 'assent', 'auxverb', 'bio', 'body', 'cause', 'certain', 'cogmech', 'conj', 'death', 'discrep', 'excl', 'family', 'feel', 'filler', 'friend', 'funct', 'future', 'health', 'hear', 'home', 'humans', 'i', 'incl', 'ingest', 'inhib', 'insight', 'ipron', 'leisure', 'money', 'motion', 'negate', 'negemo', 'nonfl', 'number', 'past', 'percept', 'posemo', 'ppron', 'preps', 'present', 'pronoun', 'quant', 'relativ', 'relig', 'sad', 'see', 'sexual', 'shehe', 'social', 'space', 'swear', 'tentat', 'they', 'time', 'verb', 'we', 'work', 'you']


### testes liwc

In [6]:
gettysburg = 'ruim'
gettysburg_tokens = tokenize(gettysburg)
# now flatmap over all the categories in all of the tokens using a generator:
gettysburg_counts = Counter(category for token in gettysburg_tokens for category in parse(token))
# and print the results:
print(gettysburg_counts)

Counter({'humans': 1, 'affect': 1, 'negemo': 1, 'anx': 1, 'anger': 1, 'cogmech': 1, 'insight': 1, 'percept': 1, 'bio': 1, 'health': 1})


In [7]:
gettysburg = 'ela é ruim, péssima'
gettysburg_tokens = tokenize(gettysburg)
cats = [category for token in gettysburg_tokens for category in parse(token)]

In [8]:
print(cats)

['funct', 'pronoun', 'ppron', 'shehe', 'ipron', 'social', 'funct', 'auxverb', 'present', 'verb', 'relativ', 'humans', 'affect', 'negemo', 'anx', 'anger', 'cogmech', 'insight', 'percept', 'bio', 'health', 'affect', 'negemo']


## Constructing liwc dataset for machine learnning

In [101]:
#os.chdir("/mydir")
f = open("data/liwccamoes.csv", "w")
f.write("id;titulo;autor;ano;genero;" + ";".join(liwcCats) + "\n"  )
id = 1
for arquivo in sorted(glob.glob("data/BDCamões_DependencyBank/*.tsv")):
    #print(arquivo)
    nomeArquivo = arquivo.split('/')[2]
    arq = nomeArquivo.split('.')[0]
    mydoc = minidom.parse("data/BDCamões/"+arq+".xml")
    #print(arq)

    items = mydoc.getElementsByTagName('title')
    title = items[0].firstChild.data

    items = mydoc.getElementsByTagName('author')
    author = items[0].firstChild.data

    items = mydoc.getElementsByTagName('other')
    other = items[0].firstChild.data
    
    for it in other.split("\n"):
        if "YY" in it:
            ano = it.split(":")[1].strip()
        if "GG" in it:
            genero = it.split(":")[1].strip()

    items = mydoc.getElementsByTagName('text')
    texto = items[0].firstChild.data
    #print(texto)
            
    texto_tokens = tokenize(texto)
    sents = [category for token in texto_tokens for category in parse(token)]
    contSents = Counter(sents).most_common()
    
    # total de tokens na obra
    num_lines = sum(1 for line in open(arquivo))-1
    
    aux = str(id)+";"+title +";"+author+";"+ano+";"+genero
    for cat in liwcCats:
        sent = [x[1] for x in contSents if x[0] == cat]
        if sent:
            valor=round(sent[0]*10000/num_lines)
        else:
            valor=0
        
        aux = aux + ";"+str(valor)
    
    f.write(aux+"\n")
    #print(aux)
    
    #if id > 3: break
    id = id + 1
f.close()

In [102]:
!wc -l data/liwccamoes.csv

196 data/liwccamoes.csv


In [103]:
!head data/liwccamoes.csv

id;titulo;autor;ano;genero;achieve;adverb;affect;anger;anx;article;assent;auxverb;bio;body;cause;certain;cogmech;conj;death;discrep;excl;family;feel;filler;friend;funct;future;health;hear;home;humans;i;incl;ingest;inhib;insight;ipron;leisure;money;motion;negate;negemo;nonfl;number;past;percept;posemo;ppron;preps;present;pronoun;quant;relativ;relig;sad;see;sexual;shehe;social;space;swear;tentat;they;time;verb;we;work;you
1;A virtude laureada;Manoel Maria de Barbosa du Bocage;1805;Drama;70;67;663;100;84;554;63;70;282;142;114;41;1295;395;35;281;303;3;106;2;12;2048;15;67;65;17;163;135;646;220;81;143;743;57;48;164;51;344;209;13;51;257;306;927;764;171;1145;51;663;83;152;77;47;545;1126;600;140;303;161;143;258;3;50;722
2;Os Pobres;Raul Brandão;1906;Novel;74;177;603;99;63;789;33;252;482;193;198;68;2209;744;31;420;460;21;131;6;44;2980;16;90;103;20;315;112;935;325;177;264;1027;87;93;255;85;319;291;105;157;356;281;1014;917;450;1381;265;1127;64;169;101;102;706;1409;819;269;623;189;323;667;3;53;763


In [104]:
!tail data/liwccamoes.csv

186;TENTAR NÃO MORRER;Gonçalo M. Tavares;2005;Tale;67;282;470;40;67;792;13;255;832;604;268;94;2497;550;0;416;644;0;40;0;27;3114;13;81;107;40;255;40;805;268;255;658;1221;94;40;430;188;161;349;67;523;765;336;1114;1020;228;1450;188;1450;134;54;470;40;993;1490;1060;188;550;188;550;819;0;54;832
187;CONFIDÊNCIA BARREIRENSE;Teresa Veiga;2000;Tale;160;260;697;87;80;846;17;339;566;279;290;95;3341;1002;17;594;678;69;163;2;59;4195;48;92;99;77;315;266;1430;510;342;482;1176;113;163;455;120;286;358;167;399;398;387;1172;1464;346;1627;369;1729;43;104;134;100;792;1701;1184;368;873;173;672;911;0;172;743
188;O ROMANCE PERDIDO;Fernando Venâncio;2004;Tale;155;330;672;105;52;754;20;260;337;157;280;115;2515;642;12;362;482;10;132;7;32;3419;20;50;87;35;255;287;1014;437;192;385;952;95;120;420;127;260;217;185;362;355;385;1021;999;290;1384;370;1406;57;87;115;57;619;1274;804;345;654;132;557;779;5;227;587
189;Auto da Barca do Inferno;Gil Vicente;1517;Drama;77;218;420;107;16;364;33;298;251;101;181;26;1502;541;36;362

In [ ]:
id_obra	ano	genero	ADJ	ADV	CARD
1	1805	Drama	579	292	17
2	1906	Novel	466	422	12
3	1892	Poetry	530	394	18
4	1914	Petry	338	450	3
5	1899	Poetry	647	276	19

In [93]:
for cat in liwcCats:
    print(cat)
    sent = [x[1] for x in contSents if x[0] == cat]
    if sent:
        print(sent[0])
    else:
        print(0)

achieve
39
adverb
45
affect
185
anger
24
anx
22
article
126
assent
5
auxverb
75
bio
115
body
58
cause
27
certain
22
cogmech
545
conj
156
death
22
discrep
97
excl
115
family
0
feel
56
filler
1
friend
11
funct
819
future
9
health
27
hear
16
home
3
humans
66
i
148
incl
261
ingest
106
inhib
48
insight
58
ipron
185
leisure
23
money
30
motion
92
negate
37
negemo
106
nonfl
49
number
22
past
40
percept
111
posemo
78
ppron
288
preps
239
present
94
pronoun
369
quant
58
relativ
362
relig
26
sad
55
see
38
sexual
20
shehe
111
social
261
space
203
swear
50
tentat
120
they
34
time
86
verb
157
we
0
work
12
you
112


In [74]:
contSents

[('funct', 819),
 ('cogmech', 545),
 ('pronoun', 369),
 ('relativ', 362),
 ('ppron', 288),
 ('social', 261),
 ('incl', 261),
 ('preps', 239),
 ('space', 203),
 ('ipron', 185),
 ('affect', 185),
 ('verb', 157),
 ('conj', 156),
 ('i', 148),
 ('article', 126),
 ('tentat', 120),
 ('excl', 115),
 ('bio', 115),
 ('you', 112),
 ('percept', 111),
 ('shehe', 111),
 ('negemo', 106),
 ('ingest', 106),
 ('discrep', 97),
 ('present', 94),
 ('motion', 92),
 ('time', 86),
 ('posemo', 78),
 ('auxverb', 75),
 ('humans', 66),
 ('insight', 58),
 ('body', 58),
 ('quant', 58),
 ('feel', 56),
 ('sad', 55),
 ('swear', 50),
 ('nonfl', 49),
 ('inhib', 48),
 ('adverb', 45),
 ('past', 40),
 ('achieve', 39),
 ('see', 38),
 ('negate', 37),
 ('they', 34),
 ('money', 30),
 ('health', 27),
 ('cause', 27),
 ('relig', 26),
 ('anger', 24),
 ('leisure', 23),
 ('anx', 22),
 ('number', 22),
 ('death', 22),
 ('certain', 22),
 ('sexual', 20),
 ('hear', 16),
 ('work', 12),
 ('friend', 11),
 ('future', 9),
 ('assent', 5),
 ('h

In [2]:
from xml.dom import minidom
import glob, os
for arquivo in glob.glob("data/BDCamões_DependencyBank/*.tsv"):
    nomeArquivo = arquivo.split('/')[2]
    arq = nomeArquivo.split('.')[0]
    mydoc = minidom.parse("data/BDCamões/"+arq+".xml")
    print(arq)
    aux = "data/BDCamões/"+arq+".xml"
    print(aux)
    break


BragaHistóriaRomantismoII
data/BDCamões/BragaHistóriaRomantismoII.xml


In [30]:
txt_tokens = tokenize(txt)
cats = [category for token in txt_tokens for category in parse(token)]
Counter(cats).most_common()

[('funct', 10), ('preps', 6), ('conj', 4), ('cogmech', 4), ('incl', 4)]

[('funct', 1389),
 ('cogmech', 1050),
 ('incl', 596),
 ('preps', 586),
 ('social', 552),
 ('relativ', 552),
 ('pronoun', 483),
 ('space', 411),
 ('ppron', 392),
 ('article', 377),
 ('you', 346),
 ('ipron', 325),
 ('conj', 301),
 ('affect', 277),
 ('shehe', 261),
 ('bio', 246),
 ('tentat', 245),
 ('verb', 221),
 ('ingest', 204),
 ('time', 177),
 ('they', 157),
 ('motion', 142),
 ('posemo', 141),
 ('percept', 139),
 ('discrep', 133),
 ('excl', 131),
 ('negemo', 130),
 ('quant', 127),
 ('swear', 127),
 ('past', 117),
 ('humans', 115),
 ('body', 108),
 ('nonfl', 95),
 ('insight', 87),
 ('cause', 84),
 ('present', 79),
 ('inhib', 71),
 ('number', 69),
 ('sexual', 67),
 ('achieve', 63),
 ('see', 61),
 ('adverb', 59),
 ('feel', 59),
 ('anger', 47),
 ('auxverb', 46),
 ('leisure', 39),
 ('work', 39),
 ('home', 38),
 ('sad', 36),
 ('anx', 30),
 ('hear', 29),
 ('money', 28),
 ('certain', 24),
 ('health', 20),
 ('friend', 18),
 ('relig', 15),
 ('family', 10),
 ('negate', 8),
 ('death', 7),
 ('futu

In [ ]:
allTxt = ""
dirs = ["/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_I/resource/", "/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_II/resource/"]
for diretorio in dirs:
    #print(diretorio)
    os.chdir(diretorio)
    for file in glob.glob("*.xml"):
        #print(file)
        mydoc = minidom.parse(diretorio+file)
        items = mydoc.getElementsByTagName('text')
        allTxt = allTxt + items[0].firstChild.data

In [23]:
dataset = {}

dirs = ["/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_I/resource/", "/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_II/resource/"]
for diretorio in dirs:
    print(diretorio)
    depdir = diretorio.replace("BDCamões_", "BDCamões_DependencyBank_")
    print(depdir)
    
    
    os.chdir(diretorio)
    for file in glob.glob("*.xml"):
        #print(file)
        arquivo = file.replace(".xml", ".tsv")
        if not os.path.isfile(depdir+arquivo):
            print("not found: ",arquivo)
            continue
        #teste = pd.read_csv(depdir+arquivo, sep='\t')
        
        doc = {}
        doc['arquivo'] = file
        #print(file)
        mydoc = minidom.parse(diretorio+file)
        
        items = mydoc.getElementsByTagName('title')
        txt = items[0].firstChild.data
        doc['titulo'] = txt

        items = mydoc.getElementsByTagName('author')
        txt = items[0].firstChild.data
        doc['autor'] = txt        

        items = mydoc.getElementsByTagName('other')
        txt = items[0].firstChild.data
        for t in txt.split("\n"):
            if t[:3] == "YY:":
                doc['ano'] = t.split(':')[1].strip()
            if t[:3] == "GG:":
                doc['genero'] = t.split(':')[1].strip()            
        
        items = mydoc.getElementsByTagName('text')
        #allTxt = allTxt + items[0].firstChild.data
        #dataset.append(doc)
        #teste['titulo'] = doc['titulo']
        #teste['genero'] = doc['genero']
        #teste['ano'] = doc['ano']
        #print(type(dfs))
        if doc['genero'] in dataset:
            dataset[doc['genero']] = dataset[doc['genero']] + "\n" + items[0].firstChild.data
        else:
            dataset[doc['genero']] = items[0].firstChild.data
        #break

/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_I/resource/
/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_DependencyBank_Part_I/resource/
not found:  21287-8.tsv
not found:  18167-8.tsv
not found:  22826-8.tsv
not found:  15674-8.tsv
not found:  11299-8.tsv
not found:  27311-8.tsv
not found:  24129-8.tsv
not found:  CamõesOsLusíadas.tsv
not found:  16571-8.tsv
not found:  16633-8.tsv
not found:  24508-8.tsv
not found:  21911-8.tsv
not found:  25987-8.tsv
/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_Part_II/resource/
/home/mateus/projetos/absa/16_Doutorado_Implicit/data/camoes/BDCamões_DependencyBank_Part_II/resource/


In [24]:
len(dataset)

16

In [25]:
len(dataset)

16

In [33]:
sents = {}
for x in dataset:
    print(x)
    txt_tokens = tokenize(dataset[x])
    sents[x] = Counter([category for token in txt_tokens for category in parse(token)]).most_common()
    #Counter(cats).most_common()

Short story
Chronicle
Tale
Drama
Novel
Poetry
Essay
Script
Allegory
Petry
Letter
Memoirs
Poem
Narrative
Other
Anthology


In [41]:
for x in sents:
    for y in sents[x]:
        print(x+";"+y[0]+";"+str(y[1]))

Short story;funct;131807
Short story;cogmech;94628
Short story;social;60919
Short story;pronoun;54378
Short story;relativ;47873
Short story;preps;45055
Short story;incl;42303
Short story;ipron;40669
Short story;ppron;39257
Short story;space;33118
Short story;verb;29385
Short story;conj;29293
Short story;you;29019
Short story;shehe;28829
Short story;article;27146
Short story;affect;23915
Short story;tentat;23050
Short story;excl;20347
Short story;discrep;17859
Short story;time;17090
Short story;bio;16769
Short story;ingest;14677
Short story;present;14170
Short story;percept;13905
Short story;motion;13429
Short story;swear;12781
Short story;posemo;12771
Short story;insight;12381
Short story;humans;11807
Short story;past;11016
Short story;auxverb;10598
Short story;negemo;10470
Short story;nonfl;10188
Short story;quant;8505
Short story;cause;8298
Short story;inhib;8124
Short story;body;8009
Short story;adverb;7897
Short story;they;7489
Short story;achieve;4903
Short story;feel;4834
Short s